# Import

In [2]:
from __future__ import annotations
from typing import Tuple, Literal, Optional
from functools import lru_cache
import os
import logging
from moviepy import VideoClip, ColorClip, VideoFileClip, ImageClip
from PIL import Image
import numpy as np
# from .geometry import fit_into_rect, warn_if_upscale

# Import Functions

In [39]:
import dataclasses

Rect = Tuple[int, int, int, int]
@dataclass(frozen=True)
class Meta:
    """Thông tin khung canvas & safe margins (tỷ lệ)."""
    width: int
    height: int
    title_area: float = 0.05   # 5% mỗi cạnh
    action_area: float = 0.025 # 2.5% mỗi cạnh


In [26]:
from typing import Literal, Tuple
import logging

# ---------- utils ----------
def _clamp(v: float, lo: float, hi: float) -> float:
    return max(lo, min(hi, v))

def _round_i(v: float) -> int:
    return int(round(v))
# ---------- 1) snap_to_safe ----------
def snap_to_safe(rect: Rect, meta: Meta) -> Rect:
    """
    Kẹp rect vào *title-safe box* theo meta.title_area.
    Lưu ý: chỉ tịnh tiến (không co giãn). Nếu rect lớn hơn vùng title-safe,
    nó sẽ bám mép trái/trên của title-safe và có thể tràn ra ngoài title-safe,
    nhưng vẫn luôn nằm *trong canvas*.

    Mẹo: nếu muốn kẹp theo action-safe, có thể truyền Meta với title_area = action_area
    (vd: Meta(w,h,title_area=meta.action_area, action_area=meta.action_area)).

    Args:
        rect: (x, y, w, h) theo px.
        meta: thông số canvas/safe.

    Returns:
        Rect đã kẹp.
    """
    x, y, w, h = rect
    cw, ch = meta.width, meta.height

    # Tính title-safe box (theo tỷ lệ biên mỗi cạnh)
    m = meta.title_area
    safe_left   = _round_i(cw * m)
    safe_right  = cw - safe_left
    safe_top    = _round_i(ch * m)
    safe_bottom = ch - safe_top

    # Nếu rect lớn hơn vùng title-safe, ta vẫn giữ w/h
    # và chỉ đảm bảo không vượt canvas
    safe_w = max(0, safe_right - safe_left)
    safe_h = max(0, safe_bottom - safe_top)

    if w > safe_w:
        # Không thể hoàn toàn nằm trong title-safe; bám mép trái title-safe
        x_new = safe_left
        # Nhưng vẫn phải đảm bảo không vượt canvas
        x_new = _round_i(_clamp(x_new, 0, cw - w))
    else:
        x_new = _round_i(_clamp(x, safe_left, safe_right - w))

    if h > safe_h:
        y_new = safe_top
        y_new = _round_i(_clamp(y_new, 0, ch - h))
    else:
        y_new = _round_i(_clamp(y, safe_top, safe_bottom - h))

    return (x_new, y_new, _round_i(w), _round_i(h))



# ---------- 2) fit_into_rect ----------
def fit_into_rect(src_w: int, src_h: int, dst_rect: Rect, mode: str = "fit") -> Rect:
    """
    Tính kích thước & vị trí *giữa* dst_rect để đặt nội dung theo tỉ lệ,
    không méo hình.

    - mode="fit": toàn bộ nội dung nhìn thấy, có thể có viền trống (contain).
    - mode="cover": lấp đầy dst_rect, có thể crop phần thừa.

    Trả về rect mới (x, y, w, h) đã căn giữa trong dst_rect.

    Lưu ý: Hàm này *không* cắt clip; việc mask/crop nên do renderer xử lý
    dựa trên dst_rect nếu mode="cover".
    """
    dx, dy, dw, dh = dst_rect
    if src_w <= 0 or src_h <= 0 or dw <= 0 or dh <= 0:
        return (dx, dy, 0, 0)

    ar_src = src_w / float(src_h)
    ar_dst = dw / float(dh)

    if mode not in ("fit", "cover"):
        mode = "fit"

    if mode == "fit":
        scale = min(dw / src_w, dh / src_h)
    else:  # cover
        scale = max(dw / src_w, dh / src_h)

    w = src_w * scale
    h = src_h * scale
    # căn giữa trong dst_rect
    x = dx + (dw - w) / 2.0
    y = dy + (dh - h) / 2.0

    # làm tròn & đảm bảo không vượt hoàn toàn khỏi canvas con
    w_i = max(1, _round_i(w))
    h_i = max(1, _round_i(h))
    x_i = _round_i(x)
    y_i = _round_i(y)

    return (x_i, y_i, w_i, h_i)


# ---------- 4) warn_if_upscale ----------
def warn_if_upscale(src_wh: Tuple[int, int], dst_rect: Rect, limit: float = 1.5) -> None:
    """
    Cảnh báo (logging.warning) nếu tỉ lệ phóng đại vượt ngưỡng.
    Dùng quy ước scale ~ max(dst_w/src_w, dst_h/src_h) (an toàn cho cả fit/cover).

    Args:
        src_wh: (src_w, src_h) kích thước nội dung gốc (px).
        dst_rect: (x, y, w, h) kích thước mục tiêu (px).
        limit: ngưỡng cảnh báo (mặc định 1.5 = 150%).
    """
    sw, sh = src_wh
    _, _, dw, dh = dst_rect
    if sw <= 0 or sh <= 0 or dw <= 0 or dh <= 0:
        return
    scale = max(dw / float(sw), dh / float(sh))
    if scale > limit:
        logging.warning(
            "Upscale factor %.2fx > %.2fx (src=%sx%s → dst=%sx%s). Nguy cơ mờ.",
            scale, limit, sw, sh, dw, dh
        )


# Utils

In [13]:
# src/tools/videos/graphics_core.py
from PIL import ImageColor  # thêm import

def convert_color_to_rgb(color) -> tuple[int, int, int]:
    """
    Chuyển mọi định dạng màu phổ biến về tuple RGB:
    - '#RRGGBB' hoặc '#RGB'
    - tên màu HTML ('red', 'white', ...)
    - tuple/list (R,G,B) hoặc (R,G,B,A) → bỏ A
    - int 0..255*255*255 → tách thành RGB
    """
    if isinstance(color, (tuple, list)):
        if len(color) == 3:
            return tuple(int(c) for c in color)
        elif len(color) == 4:  # RGBA
            return tuple(int(c) for c in color[:3])
        else:
            raise ValueError(f"Unsupported tuple/list length: {len(color)}")

    if isinstance(color, str):
        return ImageColor.getrgb(color)  # hỗ trợ hex và tên màu

    if isinstance(color, int):
        # ví dụ 0xRRGGBB
        r = (color >> 16) & 255
        g = (color >> 8) & 255
        b = color & 255
        return (r, g, b)

    raise TypeError(f"Unsupported color type: {type(color)}")

# Core

## 1. Media Probing and Loading

In [7]:
@lru_cache(maxsize=128)
def probe_media(src: str) -> dict:
    """
    Trả về thông tin cơ bản của media:
    {kind: 'image'|'video'|'svg', width, height, has_alpha, duration}.
    """
    ext = os.path.splitext(src)[1].lower()
    if ext in (".png", ".jpg", ".jpeg", ".webp"):
        im = Image.open(src)
        width, height = im.size
        has_alpha = im.mode in ("RGBA", "LA")
        return {"kind": "image", "width": width, "height": height,
                "has_alpha": has_alpha, "duration": None}
    elif ext in (".mp4", ".mov", ".mkv", ".webm", ".avi"):
        clip = VideoFileClip(src)
        return {"kind": "video", "width": clip.w, "height": clip.h,
                "has_alpha": False, "duration": clip.duration}
    elif ext == ".svg":
        # SVG sẽ xử lý ở Phase-2
        return {"kind": "svg", "width": None, "height": None,
                "has_alpha": True, "duration": None}
    else:
        raise ValueError(f"Unsupported file type: {ext}")


@lru_cache(maxsize=64)
def load_image_clip(src: str) -> ImageClip:
    """
    Nạp PNG/JPG thành ImageClip (giữ alpha nếu có).
    """
    im = Image.open(src).convert("RGBA")
    arr = np.array(im)  # numpy sẽ dùng cho ImageClip
    return ImageClip(arr, transparent=True)


def load_video_clip(src: str) -> VideoFileClip:
    """
    Nạp video thành VideoFileClip.
    Phase-2: sẽ thêm trim/loop ở Wrapper.
    """
    return VideoFileClip(src)


def rasterize_svg(src: str, dpi: int = 192) -> ImageClip:
    """
    Phase-2: Chuyển SVG thành raster ImageClip.
    (Hiện tại chưa triển khai)
    """
    raise NotImplementedError("SVG rasterization chưa hỗ trợ ở MVP.")

## 2. Transform & Quality

In [20]:
# =========================
# 1. Media probing & loading
# =========================




# =========================
# 2. Transform & Quality
# =========================

def compute_rect(src_wh: Tuple[int, int],
                 dst_rect: Rect,
                 mode: Literal["fit", "cover"] = "fit") -> Rect:
    """
    Tính rect mới cho media theo fit/cover.
    """
    return fit_into_rect(src_wh[0], src_wh[1], dst_rect, mode=mode)


def position_clip(clip: VideoClip,
                  rect: Rect,
                  opacity: float = 1.0,
                  rotation: float = 0.0) -> VideoClip:
    """
    Resize/crop clip nếu cần, set vị trí (x,y), opacity, rotation.
    """
    x, y, w, h = rect
    clip = clip.resized((w, h))
    if rotation:
        clip = clip.rotated(rotation)
    if opacity < 1.0:
        clip = clip.with_opacity(opacity)
    return clip.with_position((x, y))


def warn_if_upscale_core(src_wh: Tuple[int, int],
                         dst_rect: Rect,
                         limit: float = 1.5) -> None:
    """
    Cảnh báo upscale > limit.
    """
    warn_if_upscale(src_wh, dst_rect, limit=limit)

# =========================
# 3. Utility
# =========================

def make_solid_background(color: str|Tuple,
                           size: Tuple[int, int],
                           duration: float) -> VideoClip:
    """
    Tạo nền màu đặc với size và duration.
    """
    if not isinstance(color, (int, int,int)):
        color = convert_color_to_rgb(color)
    return ColorClip(size, color=color).with_duration(duration)


## Test

In [22]:
# tests/test_graphics_core_matrix.py
import os, logging, itertools
from typing import Tuple
import numpy as np
from PIL import Image, ImageDraw
from moviepy import CompositeVideoClip


logging.basicConfig(level=logging.INFO)

ASSETS = "assets_test_core_matrix"
os.makedirs(ASSETS, exist_ok=True)

# ---------- Helpers: make synthetic assets ----------
def make_img(path: str, size: Tuple[int,int], mode="RGBA", label=""):
    w, h = size
    bg = (30, 120, 200, 255) if mode == "RGBA" else (30, 120, 200)
    im = Image.new(mode, (w, h), bg)
    d = ImageDraw.Draw(im)
    # border
    d.rectangle([3, 3, w-4, h-4], outline=(255, 255, 255, 255) if mode=="RGBA" else (255,255,255), width=4)
    # alpha circle (chỉ PNG RGBA)
    if mode == "RGBA":
        r = min(w,h)//3
        cx, cy = w//2, h//2
        d.ellipse([cx-r, cy-r, cx+r, cy+r], fill=(255, 160, 0, 180))
    # text corner
    d.rectangle([0,0,180,36], fill=(0,0,0,160) if mode=="RGBA" else (0,0,0))
    d.text((6,8), label, fill=(255,255,255,255) if mode=="RGBA" else (255,255,255))
    im.save(path)

def ensure_assets():
    specs = [
        ("sq_small_rgba.png", (256,256), "RGBA", "SQ 256 RGBA"),
        ("sq_big_rgb.jpg",   (1600,1600), "RGB",  "SQ 1600 JPG"),
        ("wide_rgb.jpg",     (1200,600),  "RGB",  "WIDE 1200x600"),
        ("tall_rgba.png",    (600,1200),  "RGBA", "TALL 600x1200"),
        ("tiny_rgb.jpg",     (64,64),     "RGB",  "TINY 64"),
    ]
    paths = []
    for name, size, mode, label in specs:
        p = os.path.join(ASSETS, name)
        if not os.path.exists(p):
            make_img(p, size, mode, label)
        paths.append(p)
    return paths

# ---------- Test Matrix ----------
def run_matrix():
    paths = ensure_assets()
    canvas = (1600, 900)
    duration = 2.0
    bg = make_solid_background("#181C24", canvas, duration)

    # Ba “ô đặt” khác tỉ lệ để thử fit/cover
    target_rects = [
        (60,   80, 420, 420),   # square box
        (540,  80, 480, 260),   # wide banner
        (1080, 80, 360, 600),   # tall box
        (60,  520, 480, 320),
        (620, 520, 360, 360),
        (1040,520, 480, 300),
    ]

    # Kết hợp: (asset, mode, opacity, rotation)
    modes = ["fit", "cover"]
    opacities = [1.0, 0.85]
    rotations = [0, 12]

    layers = [bg]
    idx = 0
    for path, rect, mode, op, rot in itertools.islice(
        itertools.product(paths, target_rects, modes, opacities, rotations), 0, len(target_rects)*2
    ):
        info = probe_media(path)
        w, h = info["width"], info["height"]
        # Load + compute rect
        clip = load_image_clip(path)
        placed = compute_rect((w,h), rect, mode=mode)
        # Cảnh báo upscale >1.5x
        warn_if_upscale_core((w,h), placed, limit=1.5)
        # Đặt vị trí
        out = position_clip(clip, placed, opacity=op, rotation=rot)
        layers.append(out)
        idx += 1

    comp = CompositeVideoClip(layers, size=canvas).with_duration(duration)

    # Xuất 1 frame contact-sheet để rà soát nhanh (không cần render video)
    frame = comp.get_frame(0)  # numpy array HxWx3/4
    img = Image.fromarray(frame)
    out_png = os.path.join(ASSETS, "contact_sheet.png")
    img.save(out_png)
    print(f"✅ Contact-sheet saved: {out_png}")

    # Nếu muốn xem chuyển động/opacity/rotation → bật render:
    # comp.write_videofile(os.path.join(ASSETS, "matrix_preview.mp4"), fps=30, codec="libx264", audio=False)
    comp.preview()
    print("OK • Ma trận test đã chạy xong. Bật write_videofile() nếu cần video.")

if __name__ == "__main__":
    run_matrix()


✅ Contact-sheet saved: assets_test_core_matrix\contact_sheet.png
OK • Ma trận test đã chạy xong. Bật write_videofile() nếu cần video.


# Type

In [23]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Literal, Tuple, Optional

Rect = Tuple[int, int, int, int]

@dataclass
class Layout:
    rect: Rect
    mode: Literal["fit", "cover"] = "fit"
    align: Literal["left", "center", "right"] = "center"   # hiện chưa dùng ở core
    rotation: float = 0.0
    opacity: float = 1.0
    snap_safe: bool = True

@dataclass
class GraphicSpec:
    role: Literal["background", "illustration", "overlay", "special"]
    src: Optional[str] = None
    color: Optional[str] = None                 # dùng khi background/shape màu
    layout: Optional[Layout] = None
    z_hint: Optional[int] = None
    loop: bool = False                          # video (phase-2)
    trim: Optional[Tuple[float, float]] = None  # (start,dur) (phase-2)
    meta: Optional[dict] = None                 # chỗ mở rộng thêm


# Wrapper

In [27]:
from __future__ import annotations
from typing import List, Optional, Tuple
import logging

from moviepy import CompositeVideoClip
from moviepy.video.VideoClip import VideoClip



In [28]:
Clip = VideoClip

In [29]:

# =========================
# Policies & Validation
# =========================

def apply_policies(spec: GraphicSpec, meta: Meta) -> GraphicSpec:
    """
    Điền default & ép rule theo role:
    - background: default mode="cover", snap_safe=False
    - illustration: default mode="fit"
    - overlay: luôn snap_safe=True
    """
    if spec.layout is None:
        raise ValueError("GraphicSpec.layout is required")

    ly = spec.layout

    if spec.role == "background":
        if ly.mode not in ("fit", "cover"): ly.mode = "cover"
        ly.snap_safe = False
    elif spec.role == "overlay":
        ly.snap_safe = True
        if ly.mode not in ("fit", "cover"): ly.mode = "fit"
    else:  # illustration / special
        if ly.mode not in ("fit", "cover"): ly.mode = "fit"

    spec.layout = ly
    return spec

def validate_graphic_spec(spec: GraphicSpec, meta: Meta) -> List[str]:
    """
    Validate nhẹ ở wrapper: role/src/color/layout rect.
    Trả về danh sách lỗi dạng string (đủ cho MVP).
    """
    errors: List[str] = []
    if spec.role not in ("background", "illustration", "overlay", "special"):
        errors.append("role invalid")

    if spec.role == "background":
        if not spec.src and not spec.color:
            errors.append("background needs either src (image) or color")
    else:
        if not spec.src and not spec.color:
            errors.append("non-background needs src or color (shape overlay)")

    if spec.layout is None:
        errors.append("layout missing")
    else:
        x, y, w, h = spec.layout.rect
        if w <= 0 or h <= 0:
            errors.append("layout.rect width/height must be > 0")

    return errors

In [30]:
# =========================
# Builders
# =========================

def _maybe_snap(rect: Rect, spec: GraphicSpec, meta: Meta) -> Rect:
    return snap_to_safe(rect, meta) if spec.layout and spec.layout.snap_safe else rect

def build_background(spec: GraphicSpec, meta: Meta, scene_duration: float) -> Clip:
    """
    Nền màu hoặc ảnh (cover).
    """
    spec = apply_policies(spec, meta)
    errs = validate_graphic_spec(spec, meta)
    if errs:
        raise ValueError(f"Invalid background spec: {errs}")

    ly = spec.layout
    assert ly is not None
    rect = ly.rect

    if spec.color and not spec.src:
        # màu đặc
        return make_solid_background(spec.color, (meta.width, meta.height), scene_duration)

    # ảnh nền
    info = probe_media(spec.src)
    if info["kind"] != "image":
        raise ValueError(f"Background only supports images in MVP, got {info['kind']}")

    clip = load_image_clip(spec.src)
    placed = compute_rect((info["width"], info["height"]), rect, mode=ly.mode or "cover")
    warn_if_upscale_core((info["width"], info["height"]), placed, limit=1.5)
    placed = _maybe_snap(placed, spec, meta)
    return position_clip(clip, placed, opacity=ly.opacity, rotation=ly.rotation).with_duration(scene_duration)

def build_illustration(spec: GraphicSpec, meta: Meta, scene_duration: float) -> Clip:
    """
    Ảnh minh họa PNG/JPG (fit|cover), snap safe.
    """
    spec = apply_policies(spec, meta)
    errs = validate_graphic_spec(spec, meta)
    if errs:
        raise ValueError(f"Invalid illustration spec: {errs}")

    ly = spec.layout
    assert ly is not None
    rect = _maybe_snap(ly.rect, spec, meta)

    if spec.color and not spec.src:
        # shape đơn sắc (overlay dạng hình chữ nhật)
        shape = make_solid_background(spec.color, (rect[2], rect[3]), scene_duration)
        return position_clip(shape, (rect[0], rect[1], rect[2], rect[3]), opacity=ly.opacity, rotation=ly.rotation)

    info = probe_media(spec.src)
    if info["kind"] != "image":
        raise ValueError(f"Illustration supports images only in MVP, got {info['kind']}")

    clip = load_image_clip(spec.src)
    placed = compute_rect((info["width"], info["height"]), rect, mode=ly.mode)
    warn_if_upscale_core((info["width"], info["height"]), placed, limit=1.5)
    return position_clip(clip, placed, opacity=ly.opacity, rotation=ly.rotation).with_duration(scene_duration)

def build_overlay(spec: GraphicSpec, meta: Meta, scene_duration: float) -> Clip:
    """
    Logo/Watermark, ép snap_safe & z cao.
    """
    spec = apply_policies(spec, meta)
    errs = validate_graphic_spec(spec, meta)
    if errs:
        raise ValueError(f"Invalid overlay spec: {errs}")

    ly = spec.layout
    assert ly is not None
    rect = _maybe_snap(ly.rect, spec, meta)

    if spec.color and not spec.src:
        shape = make_solid_background(spec.color, (rect[2], rect[3]), scene_duration)
        return position_clip(shape, (rect[0], rect[1], rect[2], rect[3]), opacity=ly.opacity, rotation=ly.rotation)

    info = probe_media(spec.src)
    if info["kind"] != "image":
        raise ValueError(f"Overlay supports images only in MVP, got {info['kind']}")

    clip = load_image_clip(spec.src)
    placed = compute_rect((info["width"], info["height"]), rect, mode=ly.mode)
    warn_if_upscale_core((info["width"], info["height"]), placed, limit=1.5)
    return position_clip(clip, placed, opacity=ly.opacity, rotation=ly.rotation).with_duration(scene_duration)

def build_special(spec: GraphicSpec, meta: Meta, scene_duration: float) -> Optional[Clip]:
    """
    Placeholder: SVG/video alpha… chưa hỗ trợ ở MVP.
    """
    logging.warning("Special media not supported in MVP: %s", spec)
    return None


In [31]:
# =========================
# Compose
# =========================

def compose_scene(bg: Clip,
                  illustrations: list[Clip],
                  presenter: Optional[Clip],
                  overlays: list[Clip],
                  canvas_size: Tuple[int, int]) -> CompositeVideoClip:
    """
    Ghép theo thứ tự: background → illustrations → presenter → overlays.
    """
    layers: List[Clip] = []
    if bg: layers.append(bg)
    layers.extend(illustrations or [])
    if presenter: layers.append(presenter)
    layers.extend(overlays or [])
    return CompositeVideoClip(layers, size=canvas_size)


### Test

In [49]:
meta = Meta(1920,1080, title_area=0.05, action_area=0.025)
scene_dur = 3.0

# Background màu đặc full canvas
bg_spec = GraphicSpec(
    role="background",
    color="#1B1F2A",
    layout=Layout(rect=(0, 0, meta.width, meta.height), mode="cover", snap_safe=False)
)
bg = build_background(bg_spec, meta, scene_dur)

# Illustration (ảnh minh họa)
ill_spec = GraphicSpec(
    role="illustration",
    src="D:/ThienPV/code/demo/assets/img.jpg",     # đổi path ảnh thật
    layout=Layout(rect=(150, 200, 800, 600), mode="fit", opacity=0.95)
)
ill = build_illustration(ill_spec, meta, scene_dur)

# Overlay (logo/shape)
ovl_spec = GraphicSpec(
    role="overlay",
    color="yellow",   # shape trong suốt nhạt
    layout=Layout(rect=(1500, 60, 320, 120), mode="fit", opacity=1.0)
)
ovl = build_overlay(ovl_spec, meta, scene_dur)

final = compose_scene(bg, [ill], presenter=None, overlays=[ovl], canvas_size=(meta.width, meta.height))

final.preview()


In [46]:
# Nền màu full canvas 1920x1080
bg_spec = GraphicSpec(
    role="background",
    color="red",
    layout=Layout(rect=(0, 0, 1920, 1080), mode="cover", snap_safe=False)
)
bg = build_background(bg_spec, Meta(1920,1080), scene_duration=3.0)

# Nền ảnh (cover)
bg_img = GraphicSpec(
    role="background",
    src="D:/ThienPV/code/demo/assets/background.jpg",     # đổi path ảnh thật
    layout=Layout(rect=(0, 0, 1920, 1080), mode="cover", snap_safe=False)
)
bg2 = build_background(bg_img, Meta(1920,1080), scene_duration=3.0)
from moviepy import concatenate_videoclips
final = concatenate_videoclips([bg, bg2])
final.preview()

In [ ]:
# Logo PNG ở góc phải trên (fit)
ov_logo = GraphicSpec(
    role="overlay",
    src="D:/ThienPV/code/demo/assets/logo.png",
    layout=Layout(rect=(1600, 48, 240, 120), mode="fit", opacity=1.0, snap_safe=True)
)
logo = build_overlay(ov_logo, Meta(1920,1080), 3.0)

# Shape bán trong suốt làm nền cho nhãn/tiêu đề
ov_shape = GraphicSpec(
    role="overlay",
    color="#00000080",  # ~50% alpha
    layout=Layout(rect=(96, 900, 1200, 120), mode="fit", opacity=1.0, snap_safe=True)
)
title_bg = build_overlay(ov_shape, Meta(1920,1080), 3.0)

In [56]:
meta = Meta(1920,1080)
dur = 3.0

bg = build_background(bg_spec, meta, dur)
ill = build_illustration(
    GraphicSpec(role="illustration",
                src="D:/ThienPV/code/demo/assets/diagram_etl.png",
                layout=Layout(rect=(150, 220, 800, 600), mode="fit", opacity=0.95)),
    meta, dur
)
logo = build_overlay(ov_logo, meta, dur)
title_bg = build_overlay(ov_shape, meta, dur)

final = compose_scene(bg2, [ill], presenter=None, overlays=[title_bg, logo],
                      canvas_size=(meta.width, meta.height))
final.preview()